In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import jieba
import jieba.analyse

In [2]:
from bs4 import BeautifulSoup 
import requests
# 取得頁面原始碼
def getPageContent(URL):
    res = requests.get(
        url=URL,
        cookies={'over18':'1'}
    )
    content = BeautifulSoup(res.text, 'html.parser')
    return content

In [6]:
# 取得該版文章總頁數
def getPageLinks(start_page, end_page):
    #要爬的起始頁數 & 總頁數
    
    links = []
    for i in range(start_page, end_page, -1):
        link = 'https://www.ptt.cc/bbs/HatePolitics/index' + str(i) + '.html'
        links.append(link)
        
    return links

In [3]:
# 取得目前文章列表頁的文章連結
def getEachRow(URL):
    content = getPageContent(URL)
    rows = content.findAll('div',{'class':'r-ent'})
    
    link = []
    title = []
    date = []
    author = []
    push = []
    
    for row in rows:
        meta = row.find('div', 'title').find('a')
        if meta :             # 如果文章已被刪除，meta 會是 None
            link.append(meta['href'])
            title.append(meta.text)
            date.append(row.find('div', 'date').text)
            author.append(row.find('div', 'author').text)
            push.append(row.find('div', 'nrec').text)
    article_df = pd.DataFrame({'Date': date, 'Push': push, 'Title': title, 'Author': author, 'Link': link})
    return article_df

In [4]:
# 用該文章連結取得文章內文
def getArticle(URL):
    article = getPageContent(URL)
    information = article.findAll('div', ['richcontent', 'article-metaline', 'article-metaline-right', 'push'])
    information2 = article.findAll('a')
    edit = article.findAll('span', 'f2')
    #刪除不要的標籤
    for i in information:
        i = i.extract()
    for i in information2:
        i = i.extract()
    for i in edit:
        i = i.extract()
    #取得內文
    main_content = article.find(id = 'main-content')
    if main_content:
        article = main_content.text.lstrip().rstrip()
    else:
        article = 'None'
    return article

In [7]:
links = getPageLinks(3920, 3015)
all_post_meta = pd.DataFrame(columns=['Date', 'Push', 'Title', 'Author', 'Link'])
for link in links:
    df = getEachRow(link)
    df = df.iloc[::-1]
    all_post_meta = pd.concat([all_post_meta, df], ignore_index = True, sort = False)

In [8]:
#爬取內文，並併入Dataframe
contents = []
count = 0
for post_meta in all_post_meta['Link']:
    content = getArticle('https://www.ptt.cc'+ post_meta)
    contents.append(content)
    count += 1
    if count % 2500 == 0:
        print(time.ctime())

Tue Jun  4 02:06:54 2019
Tue Jun  4 02:18:43 2019
Tue Jun  4 02:30:19 2019
Tue Jun  4 02:42:25 2019
Tue Jun  4 02:54:07 2019
Tue Jun  4 03:05:46 2019
Tue Jun  4 03:17:30 2019


In [9]:
content_df = pd.DataFrame({'Content': contents})
all_post_meta = pd.concat([all_post_meta, content_df], axis = 1, ignore_index = False, sort = False) #axis = 1, 橫向合併
all_post_meta.to_csv('May_post.csv', encoding='utf_8_sig')

In [10]:
all_post_meta = pd.read_csv('May_post.csv', index_col = 0)
all_post_meta

,Date,Push,Title,Author,Link,Content
0,6/01,44,[轉錄] 吳董:韓被我這樣一搞 會變國民黨候選人,Lime5566,/bbs/HatePolitics/M.1559321142.A.26A.html,"14:50開始\r\n\r\n""韓國瑜現在被我這樣一搞，聲勢大振\r\n他這一個多月來幹一些..."
1,6/01,13,[新聞] 吳子嘉再爆：他派親信去滅證,omanorboyo,/bbs/HatePolitics/M.1559321001.A.BA0.html,指韓國瑜外遇生女挨告 吳子嘉再爆：他派親信去滅證\r\n\r\n美麗島電子報董事長吳子嘉爆料...
2,6/01,NaN,Re: [討論] 吳子嘉黑韓動機之鄙人管見,goodjeff,/bbs/HatePolitics/M.1559320847.A.CB5.html,我覺得沒什麼動機啦\r\n\r\n只是忽然想起以前一個名嘴 張友華\r\n\r\n行為模式還...
3,6/01,1,Re: [討論] 年代向錢看0531 吳董爆料真假與目的,devidevi,/bbs/HatePolitics/M.1559320749.A.8B2.html,: \r\n: 這集節目推論是說吳董爆料內容其實真假參半，當中故弄玄虛，誇大...
4,6/01,4,[討論] 有沒有關於明天凱道人數的賭盤,frenene,/bbs/HatePolitics/M.1559320536.A.99A.html,台灣人甚麼都賭\r\n\r\n請問有沒有明天凱道上人數的賭盤\r\n\r\n賭盤怎麼開??哪...
5,6/01,3,[討論] 果凍明天會上台嗎？？,eupa1973,/bbs/HatePolitics/M.1559320468.A.96E.html,凱道的大會師並沒有限制黨內的人不能出席，果凍也大器的說可以去參與，但韓粉似乎不大歡迎，你們覺...
6,6/01,11,[討論] 年代向錢看0531 吳董爆料真假與目的,loom0et0bust,/bbs/HatePolitics/M.1559320329.A.936.html,這集節目推論是說吳董爆料內容其實真假參半，當中故弄玄虛，誇大其詞的部分顯\r\n然就是想激惹...
7,6/01,15,[討論] 賀!簡書培質詢影片剛破五十萬!,LukeSkywaker,/bbs/HatePolitics/M.1559320304.A.DCD.html,狂賀! 怒賀! 太神啦\r\n\r\n簡書培議員質詢讓柯文哲拍桌暴怒的影片\r\n\r\n...
8,6/01,1,Re: [討論] 同溫層大調查,bill0205,/bbs/HatePolitics/M.1559320283.A.AF3.html,: 想要調查一下\r\n: 你的臉書好友對政治人物按讚的人數\r\n: 以柯、蔡、韓 三人比...
9,6/01,6,Re: [討論] 吳董這招厲害 中天開始打藍營,chriter,/bbs/HatePolitics/M.1559320013.A.19F.html,剛剛看新聞面對面重播也是這樣\r\n說是「自從韓的五點聲明以後\r\n權貴藍不爽到極點，才用...
